Go to [this](https://github.com/PranavEranki/Style-Transfer) link for a completed project utilizing Neural Style Transfer

If you are still confused, go to [this](https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398) link for a good explanation

# Style Transfer

During the past few years we’ve seen a slew of apps like prisma and other similar apps popping up which style your photos in a way wherein they look like paintings. Offering you a variety of beautiful styles some of which are paintings by famous artists like Starry Night by Van Gogh. Trying to explain this concept just with words might be difficult.



![](https://cdn-images-1.medium.com/max/800/0*F3xvwBKFhaQ3Mh_k)

As you can see, there are two input images, the content image and the style image, that are used to a generate a new image called the stylized image. 

A few things to notice about this image is that it has the same content as the content image and has a style similar to that of the style image. 

It looks good and we are pretty sure it’s not achieved by overlapping these two images so how do we get here - what is the math behind this idea? 

We will be using a CNN.

It’s safe to assume that a CNN does not learn to encode what the image is but it actually learns to encode what the image represents, or what contents are visible in the image and due to the inherent nonlinear nature that neural networks have, because as we go from shallow layers to deeper layers, the hidden units become capable to detect more and more complex feature from a given image.

Well this nature of encoding representations itself is the key to style transfer it is used to calculate loss between the generated image with respect to content and style image. As training the model over a ten thousands of images per class the model is able to generate similar feature representation for many different images given they belong to same class or have similar content or style. Hence it makes sense to use the difference in value of feature representation of generated image w.r.t content and style image to guide the iterations through which we produce the generated image itself but how do we make sure that content image (C) and generated image (G) are similar with respect to their content and not style, while on other hand how do we make sure that generated image only inherits similar style representation from style image (S) and not the entire style image itself. This is solved by dividing the loss function into two parts, one is the Content loss and the other is the Style loss and soon enough we will understand how they are different from each other and how they overcome the problems which we have put forth.

__TL;DR__:
We need two different loss functions for both style and content, to make sure that the generated image only inherits style representation from the style image and inherits content from the content image.

# Loss function

The loss function is one of the most important parts of the problem.

The overall loss function is:

![](https://cdn-images-1.medium.com/max/800/1*INwW0Apz4wUpDS9jetJ7xQ.jpeg)

What are the inputs of this loss function?

Alpha and Beta, the coefficients, are hyperparameters we need to optimize during the training of our program.

We randomly initialize the generated image, and slowly change it to resemble what is desired through a minimzation of the overall loss function.

![](https://cdn-images-1.medium.com/max/800/0*gq-qNqxE3b37o3dO)

During each iteration, we pass in both the content loss and the style loss.

## Content loss

To calculate this content loss, we pass both the content image and the generated image through our vgg16, and get the outputs of our model.

We find the L2 norm of element wise subtraction between these two  output matrices, as this helps to preserve the original content in the generated image by minimizing the difference in feature representation which focuses on the content difference in both images.

For a mathematical form or equation which can compute, let's use this:

![](https://cdn-images-1.medium.com/max/800/1*34xPuexhGCHT7xZ17wVvDQ.jpeg)

Where C is the content image, G is the generated image, and the layer L is the output for which we will compute loss(instead of using overall output of the VGG, we can use the output from a certain layer)

## Style Loss

__Only read this if you really want to know the math behind style loss...__

__I'm sorry if anything is confusing. These topics confuse me a lot too.__

Now let’s look at the style loss. While calculating the style loss we will consider the feature representation of many convolution layers, from shallow to deeper layers of the model. Unlike content loss, we can’t just find the difference in activation units. What we need is a way to find the correlation between these activations across different channels of the same layer. To do this we need something called as the Gram Matrix.

I will try to build up the foundation needed to understand gram matrix with an example. Let’s consider that we pass our style image though the vgg16 and we get the activation values from the 7th layer which generates a feature representation matrix of size 56x56x256.

![](https://cdn-images-1.medium.com/max/800/0*p0RFEgFg_zP-J6kD)

In this 3-D array, there are 256 channels of size 56x56 each. Now let’s assume that there is channel ‘A‘ whose activation units may get activated when they come across an image section containing black and brown strips and then there is a channel ‘B’ whose activation units may get activated when they come across something similar to an eyeball.

__Note__: Here, units getting activated refer to them having a considerably huge value compared to zero after passing through relu.

If both of these channels, ‘A’ & ‘B’, activate together for the same input, there’s a high possibility that the image might contain a face of a tiger ( because it had two channels with high values which activates for an eyeball and brown black stripes ). 

Now if both of these channels are fired up with high activation values that means they would have high correlation compared to correlation between channel ‘A’ & ‘C’ where channel ‘C’ might get activated when it sees a diamond shaped pattern.

So to get the correlation of all this channels with regard to each other we need to calculate something called as gram matrix. We will use a gram matrix to measure the degree of correlation between channels which later will act as a measure of the style itself. 

Now you might have understood the significance of gram matrix but to understand how we get the gram matrix of the above mentioned 3-D array, go through the image mentioned below.

![](https://cdn-images-1.medium.com/max/800/0*u4U60W2sblmoCtv1)

Now, as you can see, each element of this gram matrix contains the correlation measure of all the channels with respect to each other. Moving forward, how do we use this computed Gram matrix G to calculate the style loss? 
Let’s denote the gram matrix of style image of layer L as GM[L](S) and the gram matrix of a generated image of the same layer as GM[L](G). 

Both the gram matrices were computed from the same layer, and hence use the same number of channels, leading it to be a matrix of size ch x ch. Now if we find sum of square difference or L2_norm of element subtraction of these two matrices and try to minimize it, this will eventually lead to minimizing the difference between the style of style image and the generated image.

![](https://cdn-images-1.medium.com/max/800/1*IoozR3xGzaSqtEqGEKcWMQ.jpeg)

In the above equation, __N__ subscript *l* represents the number of channel in the feature-map/output of layer *l* and __M__ subscript *l* represents the height*width of the feature-map/output of layer l.

While computing style loss we use multiple activation layers, and this scenario leads us to the possibility of assigning different weightages to each sub loss provided by different layers. 

The below equation sums what I just said pretty elegantly but in our case, or most cases in general, people give equal weightage for all the layers.

![](https://cdn-images-1.medium.com/max/800/1*n7wIYY399mOdO9jJGM6aoA.jpeg)

Moving forward, once you have both content and style loss, add them up and use any optimizer to perform gradient descent to change the generated image such that it decreases its loss after each iteration.

[Link](https://arxiv.org/abs/1508.06576) to the original paper